In [72]:
%serialconnect --port=/dev/cu.usbserial-8 --baud=115200 

import machine
import display

display = display.Display()

Connecting to --port=/dev/cu.usbserial-8 --baud=115200 
Ready.


In [73]:
import sys

sys.path.append("")

from micropython import const

import uasyncio as asyncio
import aioble
from bluetooth import UUID

import random
import struct
import json

class BleClient:
    SERVER_NAME = "SnakeMultiplayer"
    SNAKE_SERVICE_UUID = UUID("ca07faee-7e95-4856-b44b-bbaef52ec7b4")
    APPLE_POSITION_CHARACTERISTIC_UUID = UUID("cec23f9f-cdc0-4577-8798-8dd4b01724d8")
    SNAKE1_POSITIONS_CHARACTERISTIC_UUID = UUID("e72c988f-7279-4b82-b808-42884a4ba48f")
    SNAKE2_POSITIONS_CHARACTERISTIC_UUID = UUID("1673be03-ef60-4fb9-869c-4513a8e212f1")
    SNAKE2_DIRECTIONS_CHARACTERISTIC_UUID = UUID("1533130f-7251-4a87-a976-1b5bc0fae798")

    transmitted_snake1_tails = []
    transmitted_snake2_tails = []
    apple_position = None

    def __init__(self) -> None:
        self.snake_service_connection = None
        self.snake1_tails_characteristic = None
        self.snake2_tails_characteristic = None
        self.snake2_direction_characteristic = None
        self.apple_position_characteristic = None

    async def read_snake1_tails(self):
        value = await self.snake1_tails_characteristic.notified()
        if (value):
            data = list(json.loads(value))
            snakeLength = int(data[0])
            newSnakeTails = data[1]
            print(data)
            for newTail in newSnakeTails:
                self.transmitted_snake1_tails.append(newTail)
            print(str(snakeLength) +" "+ str(len(self.transmitted_snake1_tails)) + " " + str(len(self.transmitted_snake1_tails) > snakeLength))
            while len(self.transmitted_snake1_tails) > snakeLength:
                self.transmitted_snake1_tails.pop(0)

        return self.transmitted_snake1_tails
    
    async def read_snake2_tails(self):
        value = await self.snake2_tails_characteristic.notified()
        if (value):
            data = list(json.loads(value))
            snakeLength = data[0]
            newSnakeTails = data[1]

            for i in range(0, len(newSnakeTails)):
                self.transmitted_snake2_tails.append(newSnakeTails[i])

            while len(self.transmitted_snake2_tails) > snakeLength:
                self.transmitted_snake2_tails.pop(0)

            print(self.transmitted_snake1_tails)

        return self.transmitted_snake2_tails
    
    async def read_apple_posistion(self):
        value = await self.apple_position_characteristic.notified()
        if (value):
            new_apple_pos = list(json.loads(value))
            print(new_apple_pos)
            self.apple_position = new_apple_pos

        return self.apple_position
    
    async def write_snake2_direction(self, newDirection: int):
        data = struct.pack("<h", newDirection)
        await self.snake2_direction_characteristic.write(data)
        print("wrote new direciton " + str(newDirection))
        
    async def find_server(self):
        if (self.snake_service_connection is not None):
            return True
        
        async with aioble.scan(5000, interval_us=30000, window_us=30000, active=True) as scanner:
            async for result in scanner:
                if result.name() == self.SERVER_NAME and self.SNAKE_SERVICE_UUID in result.services():
                    print("its a match")
                    device = result.device
                    connection = await device.connect()
                    self.snake_service_connection = await connection.service(self.SNAKE_SERVICE_UUID)
                    self.snake1_tails_characteristic = await self.snake_service_connection.characteristic(self.SNAKE1_POSITIONS_CHARACTERISTIC_UUID)
                    await self.snake1_tails_characteristic.read()
                    await self.snake1_tails_characteristic.subscribe(notify=True, indicate=False)
                    self.snake2_tails_characteristic = await self.snake_service_connection.characteristic(self.SNAKE2_POSITIONS_CHARACTERISTIC_UUID)
                    await self.snake2_tails_characteristic.subscribe(notify=True, indicate=False)
                    self.apple_position_characteristic = await self.snake_service_connection.characteristic(self.APPLE_POSITION_CHARACTERISTIC_UUID)
                    await self.apple_position_characteristic.subscribe(notify=True, indicate=False)                    
                    self.snake2_direction_characteristic = await self.snake_service_connection.characteristic(self.SNAKE2_DIRECTIONS_CHARACTERISTIC_UUID)
                    return True
            
        return False

In [74]:
BLOCK_SIZE = 4

DISPLAY_WIDTH = 128
DISPLAY_HEIGHT = 64

GAME_WIDTH = int(DISPLAY_WIDTH / BLOCK_SIZE)
GAME_HEIGHT = int(DISPLAY_HEIGHT / BLOCK_SIZE)

def draw_game(snake1, snake2, apple):
    
    display.clear()
    
    display.text("Score", 0, 0)
    for s1 in snake1:
        draw_point(s1[0], s1[1])
    
    for s2 in snake2:
        draw_point(s2[0], s2[1])

    draw_point(apple[0], apple[1])

    display.show()



def draw_point(x, y):
    display.fill_rect(x * BLOCK_SIZE, y * BLOCK_SIZE, BLOCK_SIZE, BLOCK_SIZE)



In [75]:
from time import sleep
bleClient = BleClient()

downBtn = Pin(14, Pin.IN, Pin.PULL_UP)
rightBtn = Pin(21, Pin.IN, Pin.PULL_UP)
leftBtn = Pin(18, Pin.IN, Pin.PULL_UP)
upBtn = Pin(25, Pin.IN, Pin.PULL_UP)

async def main():
    global bleClient, draw_game
    oldDirection = 1
    direction = 1

    while True:
        result = await bleClient.find_server()
        if result is False:
            continue

        if downBtn.value() == 0:
            direction = 1
        if rightBtn.value() == 0:
            direction = 3
        if leftBtn.value() == 0:
            direction = 2
        if upBtn.value() == 0:
            direction = 0

        if direction != oldDirection:
            # await bleClient.write_snake2_direction(direction)
            oldDirection = direction
    

        snake1Tails = await bleClient.read_snake1_tails()
        snake2Tails = await bleClient.read_snake2_tails()
        # applePos = await bleClient.read_apple_posistion()

        # print(applePos)
    
        draw_game(snake1Tails, snake2Tails, [0,0])

        sleep(0.02)

loop = asyncio.get_event_loop()
loop.run_until_complete(main())

its a match
[2, [[23, 10]]]
2 1 False
[[23, 10]]
[2, [[22, 10]]]
2 2 False
[[23, 10], [22, 10]]
[2, [[21, 10]]]
2 3 True
[[22, 10], [21, 10]]
[2, [[20, 10]]]
2 3 True
[[21, 10], [20, 10]]
[2, [[19, 10]]]
2 3 True
[[20, 10], [19, 10]]
[2, [[18, 10]]]
2 3 True
[[19, 10], [18, 10]]
[2, [[17, 10]]]
2 3 True
[[18, 10], [17, 10]]
[2, [[16, 10]]]
2 3 True
[[17, 10], [16, 10]]
[2, [[15, 10]]]
2 3 True
[[16, 10], [15, 10]]
[2, [[14, 10]]]
2 3 True
[[15, 10], [14, 10]]
[2, [[13, 10]]]
2 3 True
[[14, 10], [13, 10]]
[2, [[12, 10]]]
2 3 True
[[13, 10], [12, 10]]
[2, [[11, 10]]]
2 3 True
[[12, 10], [11, 10]]
[2, [[10, 10]]]
2 3 True
[[11, 10], [10, 10]]
[2, [[9, 10]]]
2 3 True
[[10, 10], [9, 10]]
[2, [[8, 10]]]
2 3 True
[[9, 10], [8, 10]]
[2, [[7, 10]]]
2 3 True
[[8, 10], [7, 10]]
[2, [[6, 10]]]
2 3 True
[[7, 10], [6, 10]]
[2, [[5, 10]]]
2 3 True
[[6, 10], [5, 10]]
[2, [[4, 10]]]
2 3 True
[[5, 10], [4, 10]]
[2, [[3, 10]]]
2 3 True
[[4, 10], [3, 10]]
[2, [[2, 10]]]
2 3 True
[[3, 10], [2, 10]]
[2, [[1

Traceback (most recent call last):
  File "<stdin>", line 44, in <module>
  File "uasyncio/core.py", line 1, in run_until_complete
  File "uasyncio/core.py", line 1, in run_until_complete
  File "uasyncio/core.py", line 1, in run_until_complete
  File "<stdin>", line 33, in main
  File "<stdin>", line 37, in read_snake1_tails
ValueError: syntax error in JSON
